# <span style='color:green'>Ottawa Neighbourhood Amenities Ranking</span>

In [2]:
# Importing necessary libraries and tools

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Completed!')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Completed!


### <span style="color:blue"> I am going to use pandas read_html function to obtain postal codes of Ottawa's neighbourhoods.  </span>

In [3]:
from pandas.io.html import read_html
wiki_page = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_K"


tables_wkp = read_html(wiki_page)

print('Extracted {num} wikitables'.format(num=len(tables_wkp)))


Extracted 5 wikitables


### It extracted 5 tables. We olnly need to use one table that include the City of Ottawa's neighbourhoods.

In [4]:
print(tables_wkp[0])

                                                    0  \
0   K1AGovernment of CanadaOttawa and Gatineau off...   
1   K1BOttawa(Blackburn Hamlet / Pine View / Sheff...   
2                                  K1COttawa(Orleans)   
3                               K1EOttawa(Queenswood)   
4   K1GOttawa(Riverview / Hawthorne / Canterbury /...   
5             K1HOttawa(Alta Vista / Billings Bridge)   
6    K1JOttawa(Beacon Hill / Cyrville / Carson Grove)   
7   K1KOttawa(Overbrook, Forbes, Manor Park, Visco...   
8                  K1LOttawa(Vanier, McKay Lake area)   
9          K1MOttawa(Rockcliffe Park / New Edinburgh)   
10  K1NOttawa(Lower Town / Byward Market / Sandy H...   
11                                K1POttawa(Downtown)   
12                          K1ROttawa(Dalhousie Ward)   
13  K1SOttawa(The Glebe / Old Ottawa South / Old O...   
14  K1TOttawa(Blossom Park / Greenboro / Leitrim) ...   
15  K1VOttawa(Heron Gate / Heron Park / Riverside ...   
16           K1WOttawa(Chapel H

In [5]:
tables_wkp[0].shape

(20, 7)

In [6]:
df0=tables_wkp[0] # Defining a pandas data frame

### <span style="color:blue"> Multiple column to single column: </span>

In [7]:
df1=pd.Series(df0.values.ravel('F'))

In [8]:
df2=pd.DataFrame(columns=['uncol'], data=df1) # Defining a pandas data frame

In [9]:
df2.shape

(140, 1)

In [10]:
df2.head()

,uncol
0,K1AGovernment of CanadaOttawa and Gatineau off...
1,K1BOttawa(Blackburn Hamlet / Pine View / Sheff...
2,K1COttawa(Orleans)
3,K1EOttawa(Queenswood)
4,K1GOttawa(Riverview / Hawthorne / Canterbury /...


### Text separation based on patterns

In [11]:
mask1 = df2['uncol'].str.contains('Not assigned')
mask2 = df2['uncol'].str.contains('Ottawa')

In [12]:
mask_pc=[]
for i in range(0, len(mask1)):
    if (mask1[i]==False) and (mask2[i]==True):
        mask_pc.append(df2.loc[i,'uncol'])

mask_pc

['K1AGovernment of CanadaOttawa and Gatineau offices (partly in QC)',
 'K1BOttawa(Blackburn Hamlet / Pine View / Sheffield Glen)',
 'K1COttawa(Orleans)',
 'K1EOttawa(Queenswood)',
 'K1GOttawa(Riverview / Hawthorne / Canterbury / Hunt Club Park)',
 'K1HOttawa(Alta Vista / Billings Bridge)',
 'K1JOttawa(Beacon Hill / Cyrville / Carson Grove)',
 'K1KOttawa(Overbrook, Forbes, Manor Park, Viscount Alexander Park, Finter Quarries)',
 'K1LOttawa(Vanier, McKay Lake area)',
 'K1MOttawa(Rockcliffe Park / New Edinburgh)',
 'K1NOttawa(Lower Town / Byward Market / Sandy Hill / University of Ottawa)',
 'K1POttawa(Downtown)',
 'K1ROttawa(Dalhousie Ward)',
 "K1SOttawa(The Glebe / Old Ottawa South / Old Ottawa East / Carleton University / Dow's Lake area)",
 'K1TOttawa(Blossom Park / Greenboro / Leitrim) / Findlay Creek',
 'K1VOttawa(Heron Gate / Heron Park / Riverside Park / Hunt Club / Riverside South / YOW)',
 'K1WOttawa(Chapel Hill South / Blackburn)',
 'K1XOttawaSouth Gloucester',
 'K1YOttawaCivic

In [13]:
len(mask_pc)

41

In [14]:
df3=pd.DataFrame(columns=['uncol'],data=mask_pc) # Defining a new pandas data frame

In [15]:
df3

,uncol
0,K1AGovernment of CanadaOttawa and Gatineau off...
1,K1BOttawa(Blackburn Hamlet / Pine View / Sheff...
2,K1COttawa(Orleans)
3,K1EOttawa(Queenswood)
4,K1GOttawa(Riverview / Hawthorne / Canterbury /...
5,K1HOttawa(Alta Vista / Billings Bridge)
6,K1JOttawa(Beacon Hill / Cyrville / Carson Grove)
7,"K1KOttawa(Overbrook, Forbes, Manor Park, Visco..."
8,"K1LOttawa(Vanier, McKay Lake area)"
9,K1MOttawa(Rockcliffe Park / New Edinburgh)


 ### Separating postal codes to a new column:

In [16]:
df3['Postcode'] = df3['uncol'].str[:3]

In [17]:
df3.head()

,uncol,Postcode
0,K1AGovernment of CanadaOttawa and Gatineau off...,K1A
1,K1BOttawa(Blackburn Hamlet / Pine View / Sheff...,K1B
2,K1COttawa(Orleans),K1C
3,K1EOttawa(Queenswood),K1E
4,K1GOttawa(Riverview / Hawthorne / Canterbury /...,K1G


### Separating neighbourhoods to a new column:

In [18]:
df3['Neighbourhood']=df3['uncol'].str[10:-1]

In [19]:
df3

,uncol,Postcode,Neighbourhood
0,K1AGovernment of CanadaOttawa and Gatineau off...,K1A,ent of CanadaOttawa and Gatineau offices (part...
1,K1BOttawa(Blackburn Hamlet / Pine View / Sheff...,K1B,Blackburn Hamlet / Pine View / Sheffield Glen
2,K1COttawa(Orleans),K1C,Orleans
3,K1EOttawa(Queenswood),K1E,Queenswood
4,K1GOttawa(Riverview / Hawthorne / Canterbury /...,K1G,Riverview / Hawthorne / Canterbury / Hunt Club...
5,K1HOttawa(Alta Vista / Billings Bridge),K1H,Alta Vista / Billings Bridge
6,K1JOttawa(Beacon Hill / Cyrville / Carson Grove),K1J,Beacon Hill / Cyrville / Carson Grove
7,"K1KOttawa(Overbrook, Forbes, Manor Park, Visco...",K1K,"Overbrook, Forbes, Manor Park, Viscount Alexan..."
8,"K1LOttawa(Vanier, McKay Lake area)",K1L,"Vanier, McKay Lake area"
9,K1MOttawa(Rockcliffe Park / New Edinburgh),K1M,Rockcliffe Park / New Edinburgh


#### Adjusting missing portions of texts

In [20]:
# adjusting missing portions of texts
df3.at[0,'Neighbourhood']='Governm'+df3.at[0,'Neighbourhood']  # adjusting missing portions of texts
df3.at[17,'Neighbourhood']='S'+df3.at[17,'Neighbourhood']
df3.at[18,'Neighbourhood']='C'+df3.at[18,'Neighbourhood']

#### Dropping the first column after extracting necessary information:

In [21]:
df3=df3.drop(columns=['uncol'])

In [22]:
df3

,Postcode,Neighbourhood
0,K1A,Government of CanadaOttawa and Gatineau office...
1,K1B,Blackburn Hamlet / Pine View / Sheffield Glen
2,K1C,Orleans
3,K1E,Queenswood
4,K1G,Riverview / Hawthorne / Canterbury / Hunt Club...
5,K1H,Alta Vista / Billings Bridge
6,K1J,Beacon Hill / Cyrville / Carson Grove
7,K1K,"Overbrook, Forbes, Manor Park, Viscount Alexan..."
8,K1L,"Vanier, McKay Lake area"
9,K1M,Rockcliffe Park / New Edinburgh


#### We need to obtain latitude and longitude for each neighbourhood. I define a user agent named ottawa_explorer. Primary source: http://download.geonames.org/export/zip/

In [23]:
da=pd.read_csv("Ottawa.txt",delimiter="\t",header=None)

In [24]:
da2=pd.DataFrame()

In [25]:
da2['pc']=da.iloc[:,1]

In [26]:
da2['la']=da.iloc[:,9]; da2['lo']=da.iloc[:,10]

In [27]:
da2['ne']=da.iloc[:,2]

In [28]:
da2

,pc,la,lo,ne
0,K1A,45.4207,-75.7023,Government of Canada Ottawa and Gatineau offices
1,K1G,45.3548,-75.5773,Ottawa (Riverview / Hawthorne)
2,K1H,45.3876,-75.6593,Ottawa (Alta Vista)
3,K1K,45.4448,-75.6431,Ottawa (Overbrook)
4,K1L,45.4400,-75.6630,Ottawa (Vanier)
5,K1M,45.4491,-75.6818,Ottawa (Rockcliffe Park / New Edinburgh)
6,K1N,45.4289,-75.6844,Ottawa (Lower Town / Sandy Hill / University o...
7,K1P,45.4225,-75.7026,Ottawa (Parliament Hill)
8,K1R,45.4123,-75.7108,Ottawa (West Downtown area)
9,K1S,45.3990,-75.6871,Ottawa (The Glebe / Ottawa South / Ottawa East)


In [29]:
da2=da2.sort_values(by='pc').reset_index(drop=True)

In [30]:
da2

,pc,la,lo,ne
0,K0A,45.2557,-76.2754,National Capital Region (Almonte)
1,K0B,45.5029,-74.7479,Prescott and Russell United Counties (Alfred)
2,K0C,45.1686,-74.8966,"Stormont, Dundas and Glengarry United Counties..."
3,K0E,44.8734,-75.4416,South Leeds and Grenville United Counties (Pre...
4,K0G,45.0466,-76.4757,Rideau Lakes area (Kemptville)
5,K0K,44.3261,-77.4318,"Quinte Shores, East Northumberland County & Pr..."
6,K0M,44.8645,-78.6553,Kawartha lakes and Haliburton County (Bobcaygeon)
7,K1A,45.4207,-75.7023,Government of Canada Ottawa and Gatineau offices
8,K1B,45.4284,-75.5875,Gloucester (Blackburn Hamlet / Pine View)
9,K1C,45.4677,-75.5399,Gloucester (West Orleans)


In [31]:
a=da2.iloc[7:49,0].reset_index(drop=True)
b=df3['Postcode']


In [32]:
tem=pd.DataFrame()

tem['c1']=a
tem['c2']=b

This location is outside of Ottawa city area:

In [33]:
da2.iloc[46,:]

pc         K4K
la     45.5501
lo     -75.291
ne    Rockland
Name: 46, dtype: object

We are droping 'Rockland' from Ottawa's neighbourhood

In [34]:
da2.drop([46],axis=0)

,pc,la,lo,ne
0,K0A,45.2557,-76.2754,National Capital Region (Almonte)
1,K0B,45.5029,-74.7479,Prescott and Russell United Counties (Alfred)
2,K0C,45.1686,-74.8966,"Stormont, Dundas and Glengarry United Counties..."
3,K0E,44.8734,-75.4416,South Leeds and Grenville United Counties (Pre...
4,K0G,45.0466,-76.4757,Rideau Lakes area (Kemptville)
5,K0K,44.3261,-77.4318,"Quinte Shores, East Northumberland County & Pr..."
6,K0M,44.8645,-78.6553,Kawartha lakes and Haliburton County (Bobcaygeon)
7,K1A,45.4207,-75.7023,Government of Canada Ottawa and Gatineau offices
8,K1B,45.4284,-75.5875,Gloucester (Blackburn Hamlet / Pine View)
9,K1C,45.4677,-75.5399,Gloucester (West Orleans)


The following steps are taken to drop other locations that are outside of Ottawa city. 

In [35]:
da2.drop([0,1,2,3,4,5,6],axis=0)

,pc,la,lo,ne
7,K1A,45.4207,-75.7023,Government of Canada Ottawa and Gatineau offices
8,K1B,45.4284,-75.5875,Gloucester (Blackburn Hamlet / Pine View)
9,K1C,45.4677,-75.5399,Gloucester (West Orleans)
10,K1E,45.4771,-75.5046,Orleans (Queenswood)
11,K1G,45.3548,-75.5773,Ottawa (Riverview / Hawthorne)
12,K1H,45.3876,-75.6593,Ottawa (Alta Vista)
13,K1J,45.4519,-75.6036,Gloucester (Beacon Hill / Cyrville)
14,K1K,45.4448,-75.6431,Ottawa (Overbrook)
15,K1L,45.4400,-75.6630,Ottawa (Vanier)
16,K1M,45.4491,-75.6818,Ottawa (Rockcliffe Park / New Edinburgh)


In [36]:
da3=da2.iloc[7:49,:]

In [37]:
da3=da3.drop([46],axis=0)

In [38]:
da3=da3.reset_index(drop=True)

In [39]:
ottawa_df=pd.DataFrame()

In [40]:
ottawa_df['Postcode']=df3['Postcode'];ottawa_df['Neighbourhood']=df3['Neighbourhood'] 

In [41]:
ottawa_df['Latitude']=da3['la'];ottawa_df['Longitude']=da3['lo'];

#### Ottawa's data frame of neighbourhoods:

In [42]:
ottawa_df

,Postcode,Neighbourhood,Latitude,Longitude
0,K1A,Government of CanadaOttawa and Gatineau office...,45.4207,-75.7023
1,K1B,Blackburn Hamlet / Pine View / Sheffield Glen,45.4284,-75.5875
2,K1C,Orleans,45.4677,-75.5399
3,K1E,Queenswood,45.4771,-75.5046
4,K1G,Riverview / Hawthorne / Canterbury / Hunt Club...,45.3548,-75.5773
5,K1H,Alta Vista / Billings Bridge,45.3876,-75.6593
6,K1J,Beacon Hill / Cyrville / Carson Grove,45.4519,-75.6036
7,K1K,"Overbrook, Forbes, Manor Park, Viscount Alexan...",45.4448,-75.6431
8,K1L,"Vanier, McKay Lake area",45.4400,-75.6630
9,K1M,Rockcliffe Park / New Edinburgh,45.4491,-75.6818


#### Ottawa City's coordinates:

In [43]:
address = 'Ottawa, Ontario'

geolocator = Nominatim(user_agent="ottawa_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Ottawa City are {}, {}.'.format(latitude, longitude))###

The geograpical coordinate of Ottawa City are 45.421106, -75.690308.


#### Ottawa's Neighbourhood map:

In [44]:
# creating map of Ottawa's Neighbourhood using latitude and longitude values
map_ottawa = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(ottawa_df['Latitude'], ottawa_df['Longitude'], ottawa_df['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        #legend_names=ottawa_df['Neighbourhood'],
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ottawa) 


map_ottawa

My Foursquare credentials (sensitive cell code):

In [45]:
# @hidden_cell
CLIENT_ID = 'HAAZJF22GFDXJR44DNU4XM3FPOSTIQGX4ZZ1DZUEMJN4HWIU' # My Foursquare ID
CLIENT_SECRET = 'WKWLPS0J1CMI4QPJWW15R0KL403GIUSCVKIT0EKSQ3VNSNKX' # My Foursquare Secret
VERSION = '20190830' # Foursquare API version

### Here is the list of selected amenities for the neighbourhoods:
School	
Library	
Park	
Medical Center	
Pharmacy	
Bus Stop	
Grocery Store	
Community Center	
Shopping Mall	
Market	
Shopping Plaza	
Coffee Shop	
Child Care Service	
Daycare	
Bank	
Post Office	
Outdoors & Recreation	
Convenience Store	
Nursery School	
Preschool	
Fast Food Restaurant	
Fire Station

### The function to acquire the amenity data set from the Foursquare API: 
#### The following function to return a data frame that include description of amenities within a neighbourhood's 1 km radius. The input is neighbourhood's geo coordinate.

#### <span style="color:blue"> Though the project indended to search every categories mentioned above, there is a limitation of daily calls on Foursquare API. Therefore, only four root categories will be searched. And then member categories will be extracted.

In [46]:
# To filter amenity-specific information, Foursquare API's category IDs are used. 
# inputs: latitude and longitude

def neigh_amen(nlat,nlon):
    
   
    venue_name=[]
    category=[]
    neighb_latitude = nlat
    neighb_longitude = nlon
    
    # ID of the carefully selected categories/amenities
    
    #category_amenities = ['4bf58dd8d48988d13b941735','4bf58dd8d48988d12f941735','4bf58dd8d48988d163941735','4bf58dd8d48988d104941735',
    #          '4bf58dd8d48988d10f951735','52f2ab2ebcbc57f1066b8b4f','4bf58dd8d48988d118951735','52e81612bcbc57f1066b7a34',
    #          '4bf58dd8d48988d1fd941735','50be8ee891d4fa8dcc7199a7','5744ccdfe4b0c0459246b4dc','4bf58dd8d48988d1e0931735',
    #          '5744ccdfe4b0c0459246b4c7','4f4532974b9074f6e4fb0104','4bf58dd8d48988d10a951735','4bf58dd8d48988d172941735',
    #          '4d4b7105d754a06377d81259','4d954b0ea243a5684a65b473','4f4533814b9074f6e4fb0107','52e81612bcbc57f1066b7a45',
    #          '4bf58dd8d48988d16e941735','4bf58dd8d48988d12c941735']
    
    
    # Due to Foursquare's daily call limitation, only four root category will be used to search venues/amenities
    # Root Categories: Professional & Other Places, Outdoors & Recreation, Shop & Service, and Food.


    category_amenities = ['4d4b7105d754a06375d81259','4d4b7105d754a06377d81259','4d4b7105d754a06378d81259','4d4b7105d754a06374d81259']
    
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    intent = 'browse'
    radius = 1000 # define radius (1 km)
    
    for cat in range(0,len(category_amenities)):
        category_ID=category_amenities[cat]
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&intent={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighb_latitude, 
        neighb_longitude, 
        category_ID,
        intent,
        radius, 
        LIMIT)
        
        results = requests.get(url).json()
        
       
        
        #if (len(results['response']['venues'])>0):
        if ('venues' in results['response']):
           
            for res in range(0,len(results['response']['venues'])):
                
                venue_name.append(results['response']['venues'][res]['name'])
                category.append(results['response']['venues'][res]['categories'][0]['name'])
    
              
    venue_amen=pd.DataFrame()
    
    venue_amen['Venue_name']=venue_name
    venue_amen['Category']=category
    
    return (venue_amen)     
    


#### To test the function's usefulness

In [131]:
am=neigh_amen(ottawa_df.at[0,'Latitude'],ottawa_df.at[0,'Longitude'])
am

,Venue_name,Category
0,Parliament Hill,Capitol Building
1,Major's Hill Park,Park
2,C.D. Howe,Office
3,Parliament of Canada - Centre Block,Capitol Building
4,Ottawa City Hall,City Hall
5,The Westin Ottawa,Hotel
6,James Michael Flaherty Building,Government Building
7,Ottawa Marriott Hotel,Hotel
8,Performance Court - 150 Elgin,Office
9,Rideau Canal,Other Great Outdoors


The function is working properly. 

#### We are now acquiring amenity data for all the neighbourhoods in Ottawa.

In [133]:
# define a data frame
amen_df=pd.DataFrame()
no_data=[] # To record neighbourhoods with no data on Foursquare


for ne in range(0,len(ottawa_df['Neighbourhood'])):

    
    temp_df=neigh_amen(ottawa_df.at[ne,'Latitude'],ottawa_df.at[ne,'Longitude'])
    
    if (temp_df.empty==False):
        
        temp_df['Neighbourhood']=ottawa_df.at[ne,'Neighbourhood']
        temp_df['Latitude']=ottawa_df.at[ne,'Latitude']
        temp_df['Longitude']=ottawa_df.at[ne,'Longitude']
        cols=['Neighbourhood','Latitude','Longitude','Venue_name','Category']
        temp_df=temp_df.reindex(columns=cols)
        amen_df=amen_df.append(temp_df)
        
    else:
        no_data.append(ne)

    
 

In [134]:
amen_df.shape

(3352, 5)

In [135]:
amen_df.head()

,Neighbourhood,Latitude,Longitude,Venue_name,Category
0,Government of CanadaOttawa and Gatineau office...,45.4207,-75.7023,Parliament Hill,Capitol Building
1,Government of CanadaOttawa and Gatineau office...,45.4207,-75.7023,Major's Hill Park,Park
2,Government of CanadaOttawa and Gatineau office...,45.4207,-75.7023,C.D. Howe,Office
3,Government of CanadaOttawa and Gatineau office...,45.4207,-75.7023,Parliament of Canada - Centre Block,Capitol Building
4,Government of CanadaOttawa and Gatineau office...,45.4207,-75.7023,Ottawa City Hall,City Hall


#### <span style="color:blue"> Unfortunately, this time no data obtained from Foursqaure for the following neighbourhoods </span> 

In [147]:
ottawa_df.loc[no_data,'Neighbourhood']

4     Riverview / Hawthorne / Canterbury / Hunt Club...
14    Blossom Park / Greenboro / Leitrim) / Findlay ...
27                            Beaverbrook / South March
35                                          North March
37                                                Navan
38                                           Cumberland
39                                             Manotick
40                                               Greely
Name: Neighbourhood, dtype: object

In [137]:
print('Total number of unique categories:',len(amen_df['Category'].unique()))

Total number of unique categories: 339


In [138]:
amen_df.groupby('Neighbourhood').count()

,Latitude,Longitude,Venue_name,Category
Neighbourhood,,,,
Alta Vista / Billings Bridge,63,63,63,63
Barrhaven,151,151,151,151
Beacon Hill / Cyrville / Carson Grove,81,81,81,81
Bells Corners / Arlington Woods/Redwood / Qualicum / Crystal Beach,19,19,19,19
Blackburn Hamlet / Pine View / Sheffield Glen,14,14,14,14
Blossom Park / Greenboro / Leitrim) / Findlay Cree,3,3,3,3
Bridlewood,105,105,105,105
Britannia /Whitehaven / Bayshore / Pinecrest,159,159,159,159
"Centrepointe, Meadowlands, City View, Craig Henry, Tangelwood, Grenfell Glen, Davidson Heights",14,14,14,14


In [139]:
# one hot encoding
ottawa_onehot = pd.get_dummies(amen_df[['Category']], prefix="", prefix_sep="")
# add neighbourhood and geo coordinate column back to dataframe
ottawa_onehot['Neighbourhood'] = amen_df['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [ottawa_onehot.columns[-1]] + list(ottawa_onehot.columns[:-1])
ottawa_onehot = ottawa_onehot[fixed_columns]

ottawa_onehot.head()

,Neighbourhood,ATM,Accessories Store,Acupuncturist,Advertising Agency,African Restaurant,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Arts & Crafts Store,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Center,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campaign Office,Campground,Candy Store,Capitol Building,Car Wash,Caribbean Restaurant,Castle,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,City,City Hall,Clothing Store,Club House,Coffee Shop,College Library,Comfort Food Restaurant,Comic Shop,Community Center,Conference Room,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Country,Courthouse,Coworking Space,Cricket Ground,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Driving School,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Entertainment Service,Event Service,Event Space,Eye Doctor,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fire Station,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flight School,Flower Shop,Food,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Funeral Home,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hospital,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Industrial Estate,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Laundromat,Laundry Service,Lawyer,Library,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Maternity Clinic,Mattress Store,Medical Center,Medical Lab,Medical Supply Store,Mediterranean Restaurant,Meeting Room,Memorial Site,Men's Store,Mental Health Office,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Mosque,Motorcycle Shop,Motorsports Shop,Mountain,Music School,Music Store,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Non-Profit,Noodle House,Nudist Beach,Nursery School,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Paintball Field,Palace,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pawn Shop,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Physical Therapist,Pizza Place,Playground,Plaza,Police Station,Pool,Portuguese Restaurant,Post Office,Professional & Other Places,Pub,Radio Station,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Recruiting Agency,Recycling Facility,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Shrine,Skate Park,Skating Rink,Ski Area,Smoke Shop,Smoo

In [140]:
ottawa_onehot.shape

(3352, 340)

#### The grouped data for each neighbourhood showing total frequency of each venue/amenity

In [141]:
ottawa_grouped = ottawa_onehot.groupby('Neighbourhood').sum().reset_index()
ottawa_grouped

,Neighbourhood,ATM,Accessories Store,Acupuncturist,Advertising Agency,African Restaurant,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Arts & Crafts Store,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Center,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campaign Office,Campground,Candy Store,Capitol Building,Car Wash,Caribbean Restaurant,Castle,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,City,City Hall,Clothing Store,Club House,Coffee Shop,College Library,Comfort Food Restaurant,Comic Shop,Community Center,Conference Room,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Country,Courthouse,Coworking Space,Cricket Ground,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Driving School,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Entertainment Service,Event Service,Event Space,Eye Doctor,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fire Station,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flight School,Flower Shop,Food,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Funeral Home,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hospital,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Industrial Estate,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Laundromat,Laundry Service,Lawyer,Library,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Maternity Clinic,Mattress Store,Medical Center,Medical Lab,Medical Supply Store,Mediterranean Restaurant,Meeting Room,Memorial Site,Men's Store,Mental Health Office,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Mosque,Motorcycle Shop,Motorsports Shop,Mountain,Music School,Music Store,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Non-Profit,Noodle House,Nudist Beach,Nursery School,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Paintball Field,Palace,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pawn Shop,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Physical Therapist,Pizza Place,Playground,Plaza,Police Station,Pool,Portuguese Restaurant,Post Office,Professional & Other Places,Pub,Radio Station,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Recruiting Agency,Recycling Facility,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Shrine,Skate Park,Skating Rink,Ski Area,Smoke Shop,Smoo

#### Getting category list

In [142]:
cat_list=list(ottawa_grouped.head(0))

#### Initial filtering of columns

In [145]:
ottawa_grouped=ottawa_grouped[['Neighbourhood',
 'Adult Education Center',
 'Assisted Living',
 'Athletics & Sports',
 'Bakery',
 'Bank',
 'Bistro',
 'Bookstore',
 'Burger Joint',
 'Bus Station',
 'Bus Stop',
 'Café',
 'Child Care Service',
 'Coffee Shop',
 'Community Center',
 'Convenience Store',
 'Cosmetics Shop',
 'Daycare',
 "Dentist's Office",
 'Discount Store',
 "Doctor's Office",
 'Dog Run',
 'Driving School',
 'Elementary School',
 'Eye Doctor',
 'Fast Food Restaurant',
 'Financial or Legal Service',
 'Fire Station',
 'Fried Chicken Joint',
 'Garden',
 'General College & University',
 'Grocery Store',
 'Gym',
 'Gym / Fitness Center',
 'Gym Pool',
 'Health & Beauty Service',
 'Health Food Store',
 'High School',
 'Hockey Arena',
 'Hospital',
 'Library',
 'Market',
 'Maternity Clinic',
 'Medical Center',
 'Medical Lab',
 'Mental Health Office',
 'Middle School',
 'Nursery School',
 'Optical Shop',
 'Outdoors & Recreation',
 'Park',
 'Pharmacy',
 'Physical Therapist',
 'Pizza Place',
 'Playground',
 'Plaza',
 'Pool',
 'Post Office',
 'Recreation Center',
 'Rehab Center',
 'Restaurant',
 'Sandwich Place',
 'School',
 'Shopping Mall',
 'Shopping Plaza',
 'Skate Park',
 'Skating Rink',
 'Ski Area',
 'Sports Club',
 'Supermarket',
 'Tennis Court',
 'Urgent Care Center',
 'Veterinarian']]


In [158]:
ottawa_grouped.shape

(33, 73)

In [159]:
ottawa_grouped.head()

,Neighbourhood,Adult Education Center,Assisted Living,Athletics & Sports,Bakery,Bank,Bistro,Bookstore,Burger Joint,Bus Station,Bus Stop,Café,Child Care Service,Coffee Shop,Community Center,Convenience Store,Cosmetics Shop,Daycare,Dentist's Office,Discount Store,Doctor's Office,Dog Run,Driving School,Elementary School,Eye Doctor,Fast Food Restaurant,Financial or Legal Service,Fire Station,Fried Chicken Joint,Garden,General College & University,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Health Food Store,High School,Hockey Arena,Hospital,Library,Market,Maternity Clinic,Medical Center,Medical Lab,Mental Health Office,Middle School,Nursery School,Optical Shop,Outdoors & Recreation,Park,Pharmacy,Physical Therapist,Pizza Place,Playground,Plaza,Pool,Post Office,Recreation Center,Rehab Center,Restaurant,Sandwich Place,School,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Sports Club,Supermarket,Tennis Court,Urgent Care Center,Veterinarian
0,Alta Vista / Billings Bridge,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,3,0,1,1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,2,1,0,0,6,0,1,0,0,0,0,0,2,1,0,0,0,0,0,0,1,0,1
1,Barrhaven,0,0,2,0,6,0,0,1,0,0,0,0,6,0,1,1,0,6,0,6,0,1,0,1,7,0,0,0,0,0,7,2,1,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,4,3,0,0,1,0,0,0,0,0,0,1,1,2,0,0,0,1,0,0,0,0,1
2,Beacon Hill / Cyrville / Carson Grove,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,4,1,0,1,0,0,4,0,2,0,0,1,0,0,0,0,2,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,4,3,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,1,1,0,1
3,Bells Corners / Arlington Woods/Redwood / Qual...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,6,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,Blackburn Hamlet / Pine View / Sheffield Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0


#### Combining columns based on categories:

In [160]:
ottawa_grouped['Food/Restaurant']=ottawa_grouped['Bakery']+ottawa_grouped['Bistro']+ottawa_grouped['Burger Joint']+ottawa_grouped['Café']+\
ottawa_grouped['Fast Food Restaurant']+ottawa_grouped['Fried Chicken Joint']+ottawa_grouped['Restaurant']+ottawa_grouped['Sandwich Place']+\
ottawa_grouped['Pizza Place']

/home/mmhasan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [164]:
ottawa_grouped=ottawa_grouped.drop(['Bakery','Bistro','Burger Joint','Café','Fast Food Restaurant','Fried Chicken Joint',\
                    'Restaurant','Sandwich Place','Pizza Place'], axis=1)

In [165]:
ottawa_grouped.shape

(33, 65)

In [166]:
ottawa_grouped['Misc.Amenities']=ottawa_grouped['Bookstore']+ottawa_grouped['Convenience Store']+ottawa_grouped['Driving School']
 

In [169]:
ottawa_grouped=ottawa_grouped.drop(['Bookstore','Convenience Store','Driving School'],axis=1)

In [170]:
ottawa_grouped.shape

(33, 63)

In [171]:
ottawa_grouped['Bus Service']=ottawa_grouped['Bus Station']+ottawa_grouped['Bus Stop']

In [174]:
ottawa_grouped=ottawa_grouped.drop(['Bus Station','Bus Stop'],axis=1)

In [175]:
ottawa_grouped.shape

(33, 62)

In [176]:
ottawa_grouped['Child Service']=ottawa_grouped['Child Care Service']+ottawa_grouped['Daycare']

In [178]:
ottawa_grouped=ottawa_grouped.drop(['Child Care Service','Daycare'],axis=1)

In [179]:
ottawa_grouped.shape

(33, 61)

In [180]:
ottawa_grouped['Beauty/Health']=ottawa_grouped['Cosmetics Shop']+ottawa_grouped['Health & Beauty Service']

In [182]:
ottawa_grouped=ottawa_grouped.drop(['Cosmetics Shop','Health & Beauty Service'],axis=1)

In [183]:
ottawa_grouped.shape

(33, 60)

In [184]:
ottawa_grouped['Dentist/Doc']=ottawa_grouped["Dentist's Office"]+ottawa_grouped["Doctor's Office"]+ottawa_grouped['Eye Doctor']

In [187]:
ottawa_grouped=ottawa_grouped.drop(["Dentist's Office", "Doctor's Office", 'Eye Doctor'],axis=1)

In [188]:
ottawa_grouped.shape

(33, 58)

In [189]:
ottawa_grouped['Schools']=ottawa_grouped['High School']+ottawa_grouped['Middle School']+ottawa_grouped['Elementary School']+ottawa_grouped['School']

In [191]:
ottawa_grouped=ottawa_grouped.drop(['High School','Middle School','Elementary School','School'],axis=1)

In [192]:
ottawa_grouped.shape

(33, 55)

In [194]:
 ottawa_grouped['Misc.Amenities']=ottawa_grouped['Misc.Amenities']+ottawa_grouped['Financial or Legal Service']+ottawa_grouped['Adult Education Center']

In [196]:
ottawa_grouped=ottawa_grouped.drop(['Financial or Legal Service','Adult Education Center'],axis=1)

In [197]:
ottawa_grouped.shape

(33, 53)

In [198]:
 ottawa_grouped['Grocery Store']=ottawa_grouped['Grocery Store']+ottawa_grouped['Health Food Store']

In [200]:
 ottawa_grouped=ottawa_grouped.drop(['Health Food Store'],axis=1)

In [201]:
ottawa_grouped.shape

(33, 52)

In [202]:
 ottawa_grouped['Gym/Outdoor']=ottawa_grouped['Gym']+ottawa_grouped['Gym / Fitness Center']+ottawa_grouped['Gym Pool']+ ottawa_grouped['Outdoors & Recreation']+\
     ottawa_grouped['Pool']+ottawa_grouped['Recreation Center']+ ottawa_grouped['Skate Park']+ottawa_grouped['Skating Rink']+\
    ottawa_grouped['Ski Area']

In [204]:
  ottawa_grouped=ottawa_grouped.drop(['Gym','Gym / Fitness Center','Gym Pool','Outdoors & Recreation','Pool','Recreation Center',\
 'Skate Park', 'Skating Rink', 'Ski Area'],axis=1)
 

In [205]:
ottawa_grouped.shape

(33, 44)

In [206]:
ottawa_grouped['Sports']=ottawa_grouped['Hockey Arena']+ottawa_grouped['Playground']+ottawa_grouped['Tennis Court']+ottawa_grouped['Sports Club']+ottawa_grouped['Athletics & Sports']

In [208]:
ottawa_grouped=ottawa_grouped.drop(['Hockey Arena','Playground','Tennis Court','Sports Club','Athletics & Sports'],axis=1)

In [209]:
ottawa_grouped.shape

(33, 40)

In [210]:
ottawa_grouped['Hospital/Clinic']=ottawa_grouped['Hospital']+ottawa_grouped['Maternity Clinic']+ottawa_grouped['Medical Center']+ottawa_grouped['Urgent Care Center']

In [212]:
ottawa_grouped=ottawa_grouped.drop(['Hospital', 'Maternity Clinic', 'Medical Center','Urgent Care Center'],axis=1)

In [213]:
ottawa_grouped.shape

(33, 37)

In [214]:
ottawa_grouped['Pet']=ottawa_grouped['Veterinarian']+ottawa_grouped['Dog Run']

In [216]:
ottawa_grouped=ottawa_grouped.drop(['Veterinarian',  'Dog Run'],axis=1)

In [217]:
ottawa_grouped.shape

(33, 36)

In [218]:
 ottawa_grouped['Shopping']=ottawa_grouped['Market']+ottawa_grouped['Plaza']+ottawa_grouped['Shopping Plaza']+ottawa_grouped['Shopping Mall']+\
    ottawa_grouped['Supermarket']+ottawa_grouped['Discount Store']

In [220]:
ottawa_grouped=ottawa_grouped.drop([ 'Market', 'Plaza','Shopping Plaza','Shopping Mall','Supermarket','Discount Store'],axis=1)

In [221]:
ottawa_grouped.shape

(33, 31)

In [222]:
ottawa_grouped['Other Medicare']=ottawa_grouped['Medical Lab']+ottawa_grouped['Mental Health Office']+ottawa_grouped['Rehab Center']+ottawa_grouped['Physical Therapist']

In [224]:
ottawa_grouped=ottawa_grouped.drop(['Medical Lab','Mental Health Office','Rehab Center', 'Physical Therapist'],axis=1)

In [225]:
ottawa_grouped.shape

(33, 28)

In [226]:
ottawa_grouped['Misc.Amenities']=ottawa_grouped['Misc.Amenities']+ottawa_grouped['Optical Shop']

In [228]:
ottawa_grouped=ottawa_grouped.drop(['Optical Shop'],axis=1)

In [229]:
ottawa_grouped.shape

(33, 27)

In [230]:
ottawa_grouped['Park']=ottawa_grouped['Park']+ottawa_grouped['Garden']

In [232]:
ottawa_grouped=ottawa_grouped.drop(['Garden'],axis=1)

In [233]:
ottawa_grouped.shape

(33, 26)

#### The final data frame:

In [69]:
ottawa_grouped

,Neighbourhood,Assisted Living,Bank,Coffee Shop,Community Center,Fire Station,General College & University,Grocery Store,Library,Nursery School,Park,Pharmacy,Post Office,Food/Restaurant,Misc.Amenities,Bus Service,Child Service,Beauty/Health,Dentist/Doc,Schools,Gym/Outdoor,Sports,Hospital/Clinic,Pet,Shopping,Other Medicare
0,Alta Vista / Billings Bridge,0,0,1,0,1,0,1,0,0,2,1,0,1,0,1,0,0,4,3,2,8,1,2,1,0
1,Barrhaven,0,6,6,0,0,0,7,0,0,4,3,0,9,2,0,0,1,13,1,4,3,2,1,2,2
2,Beacon Hill / Cyrville / Carson Grove,0,2,1,0,0,0,0,0,0,4,3,0,3,0,0,4,0,2,7,2,1,1,1,3,0
3,Bells Corners / Arlington Woods/Redwood / Qual...,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,1,3,1,2,0,0,0
4,Blackburn Hamlet / Pine View / Sheffield Glen,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,2,0,0
5,Bridlewood,0,3,2,0,0,0,1,0,2,12,2,0,3,1,3,0,0,3,10,4,1,4,2,0,0
6,Britannia /Whitehaven / Bayshore / Pinecrest,0,4,3,1,0,0,6,0,0,10,5,0,6,2,0,0,0,5,3,4,5,5,4,4,1
7,Civic Hospital / Island Park / Hintonburg / Me...,0,7,9,1,0,0,4,3,0,11,8,3,8,11,9,2,0,13,8,13,8,10,0,4,2
8,"Centrepointe, Meadowlands, City View, Craig He...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0,0,0,0
9,Centretown,0,37,25,4,0,1,11,5,4,8,15,5,23,22,14,3,0,31,11,11,0,14,2,10,1


#### Scaling of data is required since features/categories vary widely.

In [70]:
from sklearn.preprocessing import MinMaxScaler

In [73]:
ottawa_data=ottawa_grouped.drop(['Neighbourhood'],axis=1)
scale_data=MinMaxScaler()
scale_data.fit(ottawa_data)
ottawa_scaled=scale_data.transform(ottawa_data)
ottawa_scaled

array([[0.        , 0.        , 0.02564103, 0.        , 0.5       ,
        0.        , 0.08333333, 0.        , 0.        , 0.10526316,
        0.0625    , 0.        , 0.03571429, 0.        , 0.05263158,
        0.        , 0.        , 0.125     , 0.23076923, 0.15384615,
        0.88888889, 0.0625    , 0.4       , 0.09090909, 0.        ],
       [0.        , 0.16216216, 0.15384615, 0.        , 0.        ,
        0.        , 0.58333333, 0.        , 0.        , 0.21052632,
        0.1875    , 0.        , 0.32142857, 0.09090909, 0.        ,
        0.        , 0.5       , 0.40625   , 0.07692308, 0.30769231,
        0.33333333, 0.125     , 0.2       , 0.18181818, 0.5       ],
       [0.        , 0.05405405, 0.02564103, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.21052632,
        0.1875    , 0.        , 0.10714286, 0.        , 0.        ,
        1.        , 0.        , 0.0625    , 0.53846154, 0.15384615,
        0.11111111, 0.0625    , 0.2       , 0.

In [74]:
ott_rank=ottawa_scaled.transpose()
rank_df=pd.DataFrame()
rank_df['Neighbourhood']=ottawa_grouped['Neighbourhood']

In [75]:
rank_df['Amenity Score']=sum(ott_rank[:,:])

### Amenity ranking of neighbourhoods 

In [76]:
rank_a=rank_df.sort_values(by='Amenity Score', ascending=False).reset_index(drop=True)
rank_a.index+= 1 
rank_a

,Neighbourhood,Amenity Score
1,Centretown,16.658950
2,Government of CanadaOttawa and Gatineau office...,15.947554
3,Lower Town / Byward Market / Sandy Hill / Univ...,14.791857
4,Downtown,14.346140
5,Dalhousie Ward,10.434267
6,Civic Hospital / Island Park / Hintonburg / Me...,9.044369
7,The Glebe / Old Ottawa South / Old Ottawa East...,8.921706
8,Westboro / Carlington,7.982408
9,"Vanier, McKay Lake area",6.494166
10,Queenswood,5.695661


### Clustering of Neighbourhoods based on amenities:

#### How many clusters?
#### Elbow method:

In [77]:
ssd=[] # sum of square distance
K=[]   

for k in range(2,29):
    K.append(k)
    KMC=KMeans(n_clusters=k)
    KMC=KMC.fit(ottawa_scaled)
    ssd.append(KMC.inertia_)


In [78]:
import matplotlib.pyplot as plt
plt.plot(K,ssd, 'bx-' )
plt.xlabel('k')
plt.ylabel('Sum of square distance')
plt.title('Finding the best k')
plt.show()

<Figure size 640x480 with 1 Axes>

##### As the Elbow point is not clear from the plot, we will set k = 5. This is set by considering the number of neighbourhoods and the curve slope shown above.

In [79]:
# set number of clusters
kclusters = 5



# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ottawa_scaled)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 0, 3, 3, 0, 2], dtype=int32)

In [81]:
len(kmeans.labels_)

33

In [86]:
# add clustering labels

ottawa_grouped.insert(1, 'Cluster Labels', kmeans.labels_)
ottawa_grouped


,Cluster Labels,Neighbourhood,Assisted Living,Bank,Coffee Shop,Community Center,Fire Station,General College & University,Grocery Store,Library,Nursery School,Park,Pharmacy,Post Office,Food/Restaurant,Misc.Amenities,Bus Service,Child Service,Beauty/Health,Dentist/Doc,Schools,Gym/Outdoor,Sports,Hospital/Clinic,Pet,Shopping,Other Medicare
0,0,Alta Vista / Billings Bridge,0,0,1,0,1,0,1,0,0,2,1,0,1,0,1,0,0,4,3,2,8,1,2,1,0
1,3,Barrhaven,0,6,6,0,0,0,7,0,0,4,3,0,9,2,0,0,1,13,1,4,3,2,1,2,2
2,0,Beacon Hill / Cyrville / Carson Grove,0,2,1,0,0,0,0,0,0,4,3,0,3,0,0,4,0,2,7,2,1,1,1,3,0
3,0,Bells Corners / Arlington Woods/Redwood / Qual...,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,1,3,1,2,0,0,0
4,0,Blackburn Hamlet / Pine View / Sheffield Glen,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,2,0,0
5,0,Bridlewood,0,3,2,0,0,0,1,0,2,12,2,0,3,1,3,0,0,3,10,4,1,4,2,0,0
6,3,Britannia /Whitehaven / Bayshore / Pinecrest,0,4,3,1,0,0,6,0,0,10,5,0,6,2,0,0,0,5,3,4,5,5,4,4,1
7,3,Civic Hospital / Island Park / Hintonburg / Me...,0,7,9,1,0,0,4,3,0,11,8,3,8,11,9,2,0,13,8,13,8,10,0,4,2
8,0,"Centrepointe, Meadowlands, City View, Craig He...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0,0,0,0
9,2,Centretown,0,37,25,4,0,1,11,5,4,8,15,5,23,22,14,3,0,31,11,11,0,14,2,10,1


In [89]:
ottawa_merged = ottawa_df

#merge ottawa_grouped with the Ottawa data frame to add latitude/longitude for each neighbourhood
ottawa_merged = ottawa_merged.join(ottawa_grouped.set_index('Neighbourhood'), on='Neighbourhood')

ottawa_merged

,Postcode,Neighbourhood,Latitude,Longitude,Cluster Labels,Assisted Living,Bank,Coffee Shop,Community Center,Fire Station,General College & University,Grocery Store,Library,Nursery School,Park,Pharmacy,Post Office,Food/Restaurant,Misc.Amenities,Bus Service,Child Service,Beauty/Health,Dentist/Doc,Schools,Gym/Outdoor,Sports,Hospital/Clinic,Pet,Shopping,Other Medicare
0,K1A,Government of CanadaOttawa and Gatineau office...,45.4207,-75.7023,2.0,0.0,30.0,25.0,3.0,0.0,1.0,9.0,7.0,4.0,19.0,16.0,5.0,21.0,21.0,15.0,0.0,0.0,30.0,13.0,10.0,0.0,11.0,1.0,9.0,2.0
1,K1B,Blackburn Hamlet / Pine View / Sheffield Glen,45.4284,-75.5875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0
2,K1C,Orleans,45.4677,-75.5399,3.0,0.0,3.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,6.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,5.0,2.0,11.0,1.0,4.0,3.0,4.0,2.0
3,K1E,Queenswood,45.4771,-75.5046,3.0,0.0,4.0,6.0,1.0,0.0,0.0,3.0,0.0,2.0,6.0,5.0,0.0,9.0,1.0,1.0,0.0,1.0,15.0,4.0,4.0,2.0,10.0,0.0,5.0,2.0
4,K1G,Riverview / Hawthorne / Canterbury / Hunt Club...,45.3548,-75.5773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,K1H,Alta Vista / Billings Bridge,45.3876,-75.6593,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,3.0,2.0,8.0,1.0,2.0,1.0,0.0
6,K1J,Beacon Hill / Cyrville / Carson Grove,45.4519,-75.6036,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,3.0,0.0,0.0,4.0,0.0,2.0,7.0,2.0,1.0,1.0,1.0,3.0,0.0
7,K1K,"Overbrook, Forbes, Manor Park, Viscount Alexan...",45.4448,-75.6431,3.0,0.0,4.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,6.0,0.0,6.0,5.0,3.0,0.0,0.0,15.0,1.0,4.0,3.0,14.0,1.0,3.0,3.0
8,K1L,"Vanier, McKay Lake area",45.4400,-75.6630,3.0,0.0,4.0,2.0,0.0,2.0,0.0,2.0,1.0,0.0,5.0,5.0,1.0,4.0,10.0,6.0,0.0,0.0,10.0,5.0,5.0,4.0,9.0,4.0,3.0,1.0
9,K1M,Rockcliffe Park / New Edinburgh,45.4491,-75.6818,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,11.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,2.0,0.0,0.0


In [90]:
ottawa_merged=ottawa_merged.dropna() # dropping neighbourhoods whose data was not available on Foursquare

In [109]:
ottawa_merged[['Cluster Labels','Assisted Living','Bank','Coffee Shop','Community Center','Fire Station','General College & University','Grocery Store','Library','Nursery School','Park','Pharmacy','Post Office','Food/Restaurant','Misc.Amenities','Bus Service','Child Service','Beauty/Health','Dentist/Doc','Schools','Gym/Outdoor','Sports','Hospital/Clinic','Pet','Shopping','Other Medicare']]\
=ottawa_merged[['Cluster Labels','Assisted Living','Bank','Coffee Shop','Community Center','Fire Station','General College & University','Grocery Store','Library','Nursery School','Park','Pharmacy','Post Office','Food/Restaurant','Misc.Amenities','Bus Service','Child Service','Beauty/Health','Dentist/Doc','Schools','Gym/Outdoor','Sports','Hospital/Clinic','Pet','Shopping','Other Medicare']].astype(int)

/home/mmhasan/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [94]:
ottawa_merged['Cluster Labels']=ottawa_merged['Cluster Labels'].astype(int)

/home/mmhasan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### Visualizing the resulting clusters

In [101]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ottawa_merged['Latitude'], ottawa_merged['Longitude'], ottawa_merged['Neighbourhood'], ottawa_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1.9).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Cluster 1

In [110]:
ottawa_merged.loc[ottawa_merged['Cluster Labels'] == 0, ottawa_merged.columns[[1] + list(range(5, ottawa_merged.shape[1]))]]

,Neighbourhood,Assisted Living,Bank,Coffee Shop,Community Center,Fire Station,General College & University,Grocery Store,Library,Nursery School,Park,Pharmacy,Post Office,Food/Restaurant,Misc.Amenities,Bus Service,Child Service,Beauty/Health,Dentist/Doc,Schools,Gym/Outdoor,Sports,Hospital/Clinic,Pet,Shopping,Other Medicare
1,Blackburn Hamlet / Pine View / Sheffield Glen,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,2,0,0
5,Alta Vista / Billings Bridge,0,0,1,0,1,0,1,0,0,2,1,0,1,0,1,0,0,4,3,2,8,1,2,1,0
6,Beacon Hill / Cyrville / Carson Grove,0,2,1,0,0,0,0,0,0,4,3,0,3,0,0,4,0,2,7,2,1,1,1,3,0
9,Rockcliffe Park / New Edinburgh,0,1,0,0,0,0,0,2,0,11,0,1,0,0,1,0,0,1,1,0,2,0,2,0,0
15,Heron Gate / Heron Park / Riverside Park / Hun...,0,3,4,0,0,0,0,0,0,2,0,0,2,1,1,0,0,0,2,0,0,0,0,0,0
16,Chapel Hill South / Blackburn,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
17,South Glouceste,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,Eastern Nepean: Fisher Heights/ Parkwood Hills...,0,0,2,0,0,0,0,0,0,2,1,0,3,1,0,0,0,5,4,8,2,3,1,0,0
24,"Centrepointe, Meadowlands, City View, Craig He...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0,0,0,0
25,Bells Corners / Arlington Woods/Redwood / Qual...,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,1,3,1,2,0,0,0


Cluster 2

In [111]:
ottawa_merged.loc[ottawa_merged['Cluster Labels'] == 1, ottawa_merged.columns[[1] + list(range(5, ottawa_merged.shape[1]))]]

,Neighbourhood,Assisted Living,Bank,Coffee Shop,Community Center,Fire Station,General College & University,Grocery Store,Library,Nursery School,Park,Pharmacy,Post Office,Food/Restaurant,Misc.Amenities,Bus Service,Child Service,Beauty/Health,Dentist/Doc,Schools,Gym/Outdoor,Sports,Hospital/Clinic,Pet,Shopping,Other Medicare
10,Lower Town / Byward Market / Sandy Hill / Univ...,1,17,39,4,0,0,6,3,0,15,16,7,21,18,10,4,1,24,6,5,0,12,0,8,4


Cluster 3

In [112]:
ottawa_merged.loc[ottawa_merged['Cluster Labels'] == 2, ottawa_merged.columns[[1] + list(range(5, ottawa_merged.shape[1]))]]

,Neighbourhood,Assisted Living,Bank,Coffee Shop,Community Center,Fire Station,General College & University,Grocery Store,Library,Nursery School,Park,Pharmacy,Post Office,Food/Restaurant,Misc.Amenities,Bus Service,Child Service,Beauty/Health,Dentist/Doc,Schools,Gym/Outdoor,Sports,Hospital/Clinic,Pet,Shopping,Other Medicare
0,Government of CanadaOttawa and Gatineau office...,0,30,25,3,0,1,9,7,4,19,16,5,21,21,15,0,0,30,13,10,0,11,1,9,2
11,Downtown,0,31,23,2,0,0,5,8,4,17,14,3,28,21,19,0,0,32,10,9,0,7,1,11,3
30,Centretown,0,37,25,4,0,1,11,5,4,8,15,5,23,22,14,3,0,31,11,11,0,14,2,10,1


Cluster 4

In [113]:
ottawa_merged.loc[ottawa_merged['Cluster Labels'] == 3, ottawa_merged.columns[[1] + list(range(5, ottawa_merged.shape[1]))]]

,Neighbourhood,Assisted Living,Bank,Coffee Shop,Community Center,Fire Station,General College & University,Grocery Store,Library,Nursery School,Park,Pharmacy,Post Office,Food/Restaurant,Misc.Amenities,Bus Service,Child Service,Beauty/Health,Dentist/Doc,Schools,Gym/Outdoor,Sports,Hospital/Clinic,Pet,Shopping,Other Medicare
2,Orleans,0,3,1,0,1,0,3,0,0,6,0,0,4,1,1,0,0,5,2,11,1,4,3,4,2
3,Queenswood,0,4,6,1,0,0,3,0,2,6,5,0,9,1,1,0,1,15,4,4,2,10,0,5,2
7,"Overbrook, Forbes, Manor Park, Viscount Alexan...",0,4,2,0,0,0,3,0,0,4,6,0,6,5,3,0,0,15,1,4,3,14,1,3,3
8,"Vanier, McKay Lake area",0,4,2,0,2,0,2,1,0,5,5,1,4,10,6,0,0,10,5,5,4,9,4,3,1
18,Civic Hospital / Island Park / Hintonburg / Me...,0,7,9,1,0,0,4,3,0,11,8,3,8,11,9,2,0,13,8,13,8,10,0,4,2
19,Westboro / Carlington,0,6,2,0,2,0,2,3,0,4,6,1,3,0,5,4,0,28,5,2,3,16,1,2,4
20,Highland Park / McKellar Park /Westboro /Glaba...,0,2,2,0,0,0,3,0,0,9,2,1,9,1,0,0,0,12,6,9,3,7,3,2,2
21,Britannia /Whitehaven / Bayshore / Pinecrest,0,4,3,1,0,0,6,0,0,10,5,0,6,2,0,0,0,5,3,4,5,5,4,4,1
22,Queensway / Copeland Park / Central Park / Bel...,0,5,5,0,0,0,3,0,0,6,2,1,7,1,2,2,0,10,0,5,1,5,1,4,1
26,Barrhaven,0,6,6,0,0,0,7,0,0,4,3,0,9,2,0,0,1,13,1,4,3,2,1,2,2


Cluster 5

In [114]:
ottawa_merged.loc[ottawa_merged['Cluster Labels'] == 4, ottawa_merged.columns[[1] + list(range(5, ottawa_merged.shape[1]))]]

,Neighbourhood,Assisted Living,Bank,Coffee Shop,Community Center,Fire Station,General College & University,Grocery Store,Library,Nursery School,Park,Pharmacy,Post Office,Food/Restaurant,Misc.Amenities,Bus Service,Child Service,Beauty/Health,Dentist/Doc,Schools,Gym/Outdoor,Sports,Hospital/Clinic,Pet,Shopping,Other Medicare
12,Dalhousie Ward,0,4,9,3,2,0,12,2,2,16,7,1,3,11,7,0,0,19,7,13,3,12,4,2,0
13,The Glebe / Old Ottawa South / Old Ottawa East...,0,5,11,1,1,0,2,1,0,16,5,1,5,6,2,0,2,10,10,7,9,2,5,4,2


### Average Amenity Score of each cluster

In [125]:
rank_b = rank_a.join(ottawa_grouped.set_index('Neighbourhood'), on='Neighbourhood')


In [127]:
rank_cluster=rank_b[['Cluster Labels','Amenity Score']]

In [129]:
rank_cluster.groupby('Cluster Labels').mean()

,Amenity Score
Cluster Labels,
0,1.437103
1,14.791857
2,15.650881
3,5.686335
4,9.677986
